### Projects of using Spark
#### 16300200020 Yanjian Zhang

Use the linux command to delete the first 76 lines and redirect to the data_new.txt file.Then delete the last 49 lines and redirect to the data_new.txt. We can see that the total numbers of lines are 49611709

Introduce the spark package and configure the spark context.

In [1]:
import os
os.getcwd()

'/home/bigdatalab06'

In [2]:
from __future__ import print_function,division
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from itertools import islice
import re
conf = SparkConf().setAppName("turkey").setMaster("local").set("spark.executor.memory","12g")
conf.set("spark.executor.cores","4").set("spark.memory.offHeap.enabled","true")
conf.set("spark.executor.memoryOverhead","4096").set("spark.memory.offHeap.size","4096")
sc = SparkContext(conf=conf)


In [3]:
from pyspark.ml.linalg import Vector,Vectors
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import pyspark.sql as sql

data = sc.textFile('hdfs://localhost:9001/user/data_4000w.txt').map(lambda x : re.split("\t", x))
# classOfFirstName = data.map(lambda x: x[2]).distinct().persist()
# print(classOfFirstName.take(100))

In [4]:
# print(len(classOfFirstName))

In [5]:
# classOfLastName = data.map(lambda x: x[3]).distinct().persist()

In [6]:
from pyspark.sql import Row
spark = SparkSession(sc)
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer


In [7]:
currentdata = data.map(lambda x: [x[11], x[14].split()[0]])
print(currentdata.take(10))

# print(data.map(lambda x: x[14]).distinct().count()) # 207482
# print(data.map(lambda x: x[14].split()[0]).distinct().count()) # 119346
schema = ['city','street_address']
df = spark.createDataFrame(currentdata, schema)

indexer = [StringIndexer(inputCol = column, outputCol = column + "_index") for column in schema]

pipeline = Pipeline(stages = indexer)
df = pipeline.fit(df).transform(df)
df = df.drop(*schema)
df.show()


[['MALATYA', 'BOGAZICI'], ['MALATYA', 'CITILBAGI'], ['MALATYA', 'BOGAZICI'], ['MALATYA', 'KALE'], ['MALATYA', 'CITILBAGI'], ['MALATYA', 'CITILBAGI'], ['MALATYA', 'KALE'], ['MALATYA', 'KALE'], ['MALATYA', 'KALE'], ['MALATYA', 'BOGAZICI']]
+----------+--------------------+
|city_index|street_address_index|
+----------+--------------------+
|      24.0|               809.0|
|      24.0|             92338.0|
|      24.0|               809.0|
|      24.0|                79.0|
|      24.0|             92338.0|
|      24.0|             92338.0|
|      24.0|                79.0|
|      24.0|                79.0|
|      24.0|                79.0|
|      24.0|               809.0|
|      24.0|                79.0|
|      24.0|                79.0|
|      24.0|                79.0|
|      24.0|                79.0|
|      24.0|               809.0|
|      24.0|               809.0|
|      24.0|             92338.0|
|      24.0|                79.0|
|      24.0|             92338.0|
|      24.0|  

In [8]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
vectorForm = df.rdd.map(lambda row: (row[0], Vectors.dense(row[1:2])))
Vectordf = spark.createDataFrame(vectorForm, ['label', 'features'])
trainData, validData, testData = Vectordf.rdd.randomSplit([0.7,0.1,0.2])
# randomforest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=3)
rfmodel = rf.fit(trainData.toDF())
predictions = rfmodel.transform(testData.toDF())

p1 = predictions.select('probability', 'label')

In [9]:
p1 = predictions.select('probability', 'label')

In [10]:
p1.show(30)

+--------------------+-----+
|         probability|label|
+--------------------+-----+
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.06093868190131...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
|[0.28781466500985...| 24.0|
+--------------------+-----+
only showing t

In [12]:
# print(p1.rdd.map(lambda x: (list(enumerate(x[0])), x[1])).take(10))
hits = p1.rdd.map(lambda x: (sorted(enumerate(x[0]), key = lambda y: -y[1]), x[1])).map(lambda row: ([x[0] for x in row[0]], row[1])) # 按index 排序
hits = hits.map(lambda x: [ 1 if x[0][i]== x[1] else 0 for i in range(5)])
total = hits.count()
hits = [hits.filter(lambda x: x[i] == 1).count()/total for i in range(5)]
print(hits)
# hits.take(2000)

[0.22996436894098976, 0.07439285909703836, 0.04869332379240792, 0.04343798619180908, 0.038241862958964444]


In [ ]:
# topn = [p1.rdd.map(lambda x: (hit_n(x[0],1),x[1])).filter(lambda x: int(x[1]) in x[0]).count()/float(top1_pre.count()) for i]

In [ ]:
# print(top1_pre.take(10))
# print(top1_pre.filter(lambda x: int(x[1]) in x[0]).count()/float(top1_pre.count()))